In [ ]:
# to do - 3d convex hull around neuron cells 
#https://stackoverflow.com/questions/46310603/how-to-compute-convex-hull-image-volume-in-3d-numpy-arrays
#things get labelled so many times for different purposes ... silly

In [ ]:
import torch
import gc
# torch.cuda.empty_cache()
# gc.collect()

In [ ]:
import os
import numpy as np
from skimage import exposure, img_as_ubyte,filters
import scipy.ndimage

def Threshold(input_img, Th_value):
    tmp = np.zeros(input_img.shape, dtype=bool)
    if isinstance(Th_value, int):
        tmp[input_img == Th_value] = 1
    else:
        if isinstance(Th_value, float):
            tmp[input_img > 0. & input_img < 1.] = 1
        else:
            for th_val in range(len(Th_value)):
                tmp[input_img == Th_value[th_val]] = 1
    return tmp


from scipy.ndimage import zoom

In [ ]:
#!!nvidia-smi

In [ ]:
import os
os.chdir('F:\\GRETA\\') #in theory we only need to change these two paths and just copy over the data 
base_path= 'F:\\GRETA\\'

In [ ]:
from pathlib import Path
import pandas as pd
rootdir = Path(base_path+("images\\")) #!!!!!!!!!!!!!!!!!!!!!
file_list = [f for f in rootdir.resolve().glob('**/*') if f.is_file()]
file_list=pd.DataFrame(data=file_list)
file_list.columns = ['Data_Path']
#file_list. tail(-1) ######!!!!!!!!!!!!!!!
file_list = file_list.map(str)
file_list['TEXT_1'] = file_list['Data_Path'].str.split('images').str[1]
file_list['TEXT_1'] = file_list['TEXT_1'].str.replace('\\', '')
OME_df = file_list[file_list['TEXT_1'].str.contains("format.tif")]
MASK_df = file_list[file_list['TEXT_1'].str.contains("mask.tif")]
Outline_df=file_list[file_list['TEXT_1'].str.contains("outline.tif")]

OME_df=OME_df.assign(IMAGE_ID=OME_df['TEXT_1'].str.extract('(.*)_Tiff_format'))
MASK_df=MASK_df.assign(IMAGE_ID=MASK_df['TEXT_1'].str.extract('(.*)_mask'))
Outline_df=Outline_df.assign(IMAGE_ID=Outline_df['TEXT_1'].str.extract('(.*)_neuron_outline'))

In [ ]:
threshold_df = pd.read_csv(base_path+'sod_ref_master_batch1.csv')
threshold_df=threshold_df.assign(IMAGE_ID=threshold_df['IMAGE_ID'].str.extract('(.*)_Tiff_format'))
#threshold_df=threshold_df.assign(IMAGE_ID=threshold_df['Image name'].str.extract('(.*)_p'))
OME_df= pd.merge(threshold_df, OME_df, on="IMAGE_ID")

In [ ]:
len(OME_df)

In [ ]:
UNIQUE_ID =np.unique(OME_df['IMAGE_ID'].astype(str))
len(UNIQUE_ID)

In [ ]:
#UNIQUE_ID

In [ ]:
out='F:\\GRETA\OUT\\'

In [ ]:
#problem cells
##############################################
#MS113_SN_Z5_2_GS g cell dont segment anything (empty mesh error)
#UNIQUE_ID=UNIQUE_ID[UNIQUE_ID == "MS116_SN_Z1_1_GS"]
#########################################################################
len(UNIQUE_ID)

In [ ]:
import tifffile 
import matplotlib.pyplot as plt
from cellpose import models, core
import skimage.io
use_GPU = core.use_gpu()
from skimage.morphology import remove_small_objects
from skimage.measure import label, regionprops, regionprops_table
from skimage.filters import threshold_otsu, threshold_yen ,threshold_triangle, threshold_li
#from skimage.morphology import dilation
from scipy import ndimage as ndi
from vedo import *
import pyvista as pv

out_render='F:\\GRETA\\3D_RENDERS_OUT\\'
cellpose_intrem='F:\\GRETA\\CELLPOSE_INTREM\\'

for i in UNIQUE_ID:
    ID_BOOL=OME_df.IMAGE_ID ==  i
    SUB_DF = OME_df[ID_BOOL]
    SUB_DF = SUB_DF.drop_duplicates(subset=['IMAGE_ID'])
    OME_path=(SUB_DF.Data_Path)
    OME_path=np.array(OME_path)
    print (OME_path)
    print (i)
    img=tifffile.imread(OME_path[0])
    
    X_PIXEL_SIZE =SUB_DF.X_pix*4 #because using 2nd scene
    Y_PIXEL_SIZE =SUB_DF.Y_pix*4 #
    Z_PIXEL_SIZE =0.5
    
    start_slice=int(SUB_DF.first_slice)
    last_slice=int(SUB_DF.last_slice)

    protein=(img[start_slice:last_slice, 1,:,:]) 
    #dapi=(img[start_slice:last_slice, 0,:,:]) 
    cells=(img[start_slice:last_slice, 2,:,:]) 
    gilal_cells=(img[start_slice:last_slice, 3,:,:]) 
    
    skimage.io.imsave(out+i+"_protein.tif", (protein))
    skimage.io.imsave(out+i+"_cells.tif", (cells))
    skimage.io.imsave(out+i+"_g_cells.tif", (gilal_cells))
    
    #load mask
    ID_BOOL_mask=MASK_df.IMAGE_ID ==  i
    SUB_DF_mask = MASK_df[ID_BOOL_mask]
    SUB_DF_mask = SUB_DF_mask.drop_duplicates(subset=['IMAGE_ID'])
    mask_path=(SUB_DF_mask.Data_Path)
    mask_path=np.array(mask_path)
    mask=tifffile.imread(mask_path[0])
    mask=(mask[start_slice:last_slice, 0,:,:]) 
    
    #load outline
    ID_BOOL_outline=Outline_df.IMAGE_ID ==  i
    SUB_DF_outline = Outline_df[ID_BOOL_outline]
    SUB_DF_outline= SUB_DF_outline.drop_duplicates(subset=['IMAGE_ID'])
    outline_path=(SUB_DF_outline.Data_Path)
    outline_path=np.array(outline_path)
    cell_outline=tifffile.imread(outline_path[0])
    cell_outline=(cell_outline[start_slice:last_slice, 0,:,:]) 
    
#     plt.figure(figsize= (5,5));

#     plt.subplot(2,2,1);
#     plt.imshow(protein[20], cmap="gray");
#     plt.title("protein");

#     plt.subplot(2,2,2);
#     plt.imshow(mask[20], cmap="gray");
#     plt.title("mask" );

#     plt.subplot(2,2,3);
#     plt.imshow(cells[20], cmap="gray");
#     plt.title("cells" );

#     plt.subplot(2,2,4);
#     plt.imshow(gilal_cells[20], cmap="gray");
#     plt.title("gilal_cells" );
    
#     plt.show();
    

    #Segment Cells using Cellpose
    #Using CYTO2
    #model = models.Cellpose(gpu=use_GPU, model_type='cyto2')
    #channels = [0,0]
    #cell_AI, flows, styles, diams = model.eval(cells,flow_threshold=None, channels=channels, do_3D=False, stitch_threshold=0.25)
    #Using newere models
    
    #use trained model from gretas segmentation 
    
    model_path='F:\\GRETA_deprec\\models_out\\from_scratch_lr0.1_500_epoch\\cellpose_residual_on_style_on_concatenation_off_train_folder_2024_04_11_16_19_30.312228'
    model = models.CellposeModel(pretrained_model=model_path, gpu=False)
    if os.path.isfile(cellpose_intrem+i+"cell_AI.tif"):
                print('cellpose already run')
                #cell_AI = skimage.io.imread(cellpose_intrem+i+"cell_AI.tif")
    else:  
                from numba import cuda 
                torch.cuda.empty_cache()
                gc.collect()
                
                #cells=zoom(cells, (0.5, 0.5,0.5)) #sml NEED BIGGER GPU 
                cell_AI, flows, styles = model.eval(cells,flow_threshold=None, do_3D=False, stitch_threshold=0.25)
                skimage.io.imsave(cellpose_intrem+i+"cell_AI.tif", (cell_AI))
                #cell_AI=zoom(cell_AI, (2, 2,2)) #sml
                
                
    #count test            
#     mid_slice=int(len(cell_AI)/2)
#     print("number of cells")
#     print(len(np.unique((cell_outline *cell_AI)[mid_slice])))
    
#     skimage.io.imsave(out+i+"cell_AI_mid_slice.tif", (cell_outline *cell_AI)[mid_slice])
#     skimage.io.imsave(out+i+"cell_mid_slice.tif", (cells)[mid_slice])
    
    if os.path.isfile(cellpose_intrem+i+"cell_cleaned.tif"):
                print('loading cleaned cell image')
                CELL_binary = skimage.io.imread(cellpose_intrem+i+"cell_cleaned.tif")
    else: 
                cell_AI= (cell_AI>1).astype(int)
                cell_AI=scipy.ndimage.binary_dilation(cell_AI)
                cell_AI=scipy.ndimage.binary_fill_holes(cell_AI)
                cell_AI=scipy.ndimage.binary_erosion(cell_AI)
                cell_AI_label=label(cell_AI)
                cell_AI=remove_small_objects(cell_AI_label, min_size=9500, connectivity=3)
                CELL_binary= (cell_AI>1).astype(int)
                skimage.io.imsave(cellpose_intrem+i+"cell_cleaned.tif", (CELL_binary))
                del (cell_AI)

    #Segment gcells
    gilal_cells_raw_masked=img_as_ubyte(mask * gilal_cells)
    thresh_gcell_number = threshold_yen(gilal_cells_raw_masked)
    g_CELL_binary = gilal_cells_raw_masked > (thresh_gcell_number)
    g_CELL_label=label(g_CELL_binary)
    g_CELL_binary = remove_small_objects(g_CELL_label, min_size=1600, connectivity=3)
    g_CELL_binary= (g_CELL_binary>1).astype(int)
    
    skimage.io.imsave(out+i+"_g_cells_binary.tif", (g_CELL_binary))
    
    
    #Seg sod 1
    min_thresh=int(SUB_DF.min_thresh)
    max_thresh=int(SUB_DF.max_thresh)
    protein_range = list(range(min_thresh, max_thresh, 1))
    protein_seg =Threshold(protein, (protein_range)).astype('uint8') 
    
    #Put everything in mask
    
    gilal_cells_masked = mask * g_CELL_binary
    cells_masked = cell_outline * CELL_binary
    protein_mask= mask * protein_seg
    
#     #outside_cell_area #THIS IS DEPREC BECAUSE GRETA HAND DRAWS THIS NOW

#     i_step=50

#     diamond = ndi.generate_binary_structure(rank=3, connectivity=1)
#     iternation_n=i_step
#     cells_masked = (cells_masked != 0).astype(bool)
#     dilated = ndi.binary_dilation(cells_masked, diamond, iterations=iternation_n)
#     outside_cell_area= cells_masked ^ dilated
#     outside_cell_area = mask * outside_cell_area

#     #outside g cell area
#     diamond = ndi.generate_binary_structure(rank=3, connectivity=1)
#     iternation_n=i_step
#     gilal_cells_masked = (gilal_cells_masked != 0).astype(bool)
#     dilated = ndi.binary_dilation(gilal_cells_masked, diamond, iterations=iternation_n)
#     outside_g_cell_area= gilal_cells_masked ^ dilated
#     outside_g_cell_area = mask * outside_g_cell_area
    
    #Vis
    g_CELL_VOL = Volume(gilal_cells_masked, spacing=(X_PIXEL_SIZE,Y_PIXEL_SIZE,Z_PIXEL_SIZE)) #sml
    g_CELL_ISO = g_CELL_VOL.isosurface(1)
    g_CELL_ISO.write(out+i+"_g_CELL.stl")
    g_Cell_STL=pv.read(out+i+"_g_CELL.stl")

    PROTEIN_VOL = Volume(protein_mask, spacing=(X_PIXEL_SIZE,Y_PIXEL_SIZE,Z_PIXEL_SIZE)) #sml
    PROTEIN_ISO = PROTEIN_VOL.isosurface(1)
    PROTEIN_ISO.write(out+i+"_PROTEIN.stl")
    PROTEIN_STL=pv.read(out+i+"_PROTEIN.stl")

    CELL_VOL = Volume(cells_masked, spacing=(X_PIXEL_SIZE,Y_PIXEL_SIZE,Z_PIXEL_SIZE)) #sml
    CELL_ISO = CELL_VOL.isosurface(1)
    CELL_ISO.write(out+i+"_CELL.stl")
    CELL_STL=pv.read(out+i+"_CELL.stl")

#     outside_cell_area_VOL = Volume(outside_cell_area, spacing=(X_PIXEL_SIZE,Y_PIXEL_SIZE,Z_PIXEL_SIZE)) #sml
#     outside_cell_area_ISO = outside_cell_area_VOL.isosurface(1)
#     outside_cell_area_ISO.write(out+i+"_outside_cell_area.stl")
#     outside_cell_area_STL=pv.read(out+i+"_outside_cell_area.stl")

#     outside_g_cell_area_VOL = Volume(outside_g_cell_area, spacing=(X_PIXEL_SIZE,Y_PIXEL_SIZE,Z_PIXEL_SIZE)) #sml
#     outside_g_cell_area_ISO = outside_g_cell_area_VOL.isosurface(1)
#     outside_g_cell_area_ISO.write(out+i+"_outside_g_cell_area.stl")
#     outside_g_cell_area_STL=pv.read(out+i+"_outside_g_cell_area.stl")

    mask_VOL = Volume(mask, spacing=(X_PIXEL_SIZE,Y_PIXEL_SIZE,Z_PIXEL_SIZE)) #sml
    mask_ISO = mask_VOL.isosurface(1)
    mask_ISO.write(out+i+"_mask.stl")
    mask_area_STL=pv.read(out+i+"_mask.stl")
    
    outline_VOL = Volume(cell_outline, spacing=(X_PIXEL_SIZE,Y_PIXEL_SIZE,Z_PIXEL_SIZE)) #sml
    outline_ISO = outline_VOL.isosurface(1)
    outline_ISO.write(out+i+"_outline.stl")
    outline_area_STL=pv.read(out+i+"_outline.stl")
    
    
    if g_Cell_STL.n_points == 0:
        pv.set_plot_theme("document");
        p = pv.Plotter();
        #p.add_mesh(g_Cell_STL, color="yellow", opacity=0.5);
        p.add_mesh(CELL_STL, color="green", opacity=0.5);
        p.add_mesh(PROTEIN_STL, color="red", opacity=1);
        #p.add_mesh(outside_cell_area_STL, color="pink", opacity=0.3);
        #p.add_mesh(outside_g_cell_area_STL, color="grey", opacity=0.3);
        p.add_mesh(mask_area_STL, color="black", opacity=1);
        p.add_mesh(outline_area_STL, color="grey", opacity=1);
        p.window_size = 1500, 1500;
        p.show(jupyter_backend='static');
        p.screenshot(out_render+i+"_3D_RENDER.png") ;
        
    elif PROTEIN_STL.n_points == 0:
        pv.set_plot_theme("document");
        p = pv.Plotter();
        p.add_mesh(g_Cell_STL, color="yellow", opacity=0.5);
        p.add_mesh(CELL_STL, color="green", opacity=0.5);
        #p.add_mesh(PROTEIN_STL, color="red", opacity=1);
        #p.add_mesh(outside_cell_area_STL, color="pink", opacity=0.3);
        #p.add_mesh(outside_g_cell_area_STL, color="grey", opacity=0.3);
        p.add_mesh(mask_area_STL, color="black", opacity=1);
        p.add_mesh(outline_area_STL, color="grey", opacity=1);
        p.window_size = 1500, 1500;
        p.show(jupyter_backend='static');
        p.screenshot(out_render+i+"_3D_RENDER.png") ;
    else:
        pv.set_plot_theme("document");
        p = pv.Plotter();
        p.add_mesh(g_Cell_STL, color="yellow", opacity=0.5);
        p.add_mesh(CELL_STL, color="green", opacity=0.5);
        p.add_mesh(PROTEIN_STL, color="red", opacity=1);
        #p.add_mesh(outside_cell_area_STL, color="pink", opacity=0.3);
        #p.add_mesh(outside_g_cell_area_STL, color="grey", opacity=0.3);
        p.add_mesh(mask_area_STL, color="black", opacity=1);
        p.add_mesh(outline_area_STL, color="grey", opacity=1);
        p.window_size = 1500, 1500;
        p.show(jupyter_backend='static');
        p.screenshot(out_render+i+"_3D_RENDER.png") ;

    
    void_binary=gilal_cells_masked.copy() + cells_masked.copy()
    void_binary = (void_binary == 0).astype(np.uint8)
    void_binary = mask * void_binary
    
    #Get information #this should be a function 
    protein_in_void = void_binary * protein_seg
    protein_in_g_cell = gilal_cells_masked * protein_seg
    protein_in_cell = cells_masked * protein_seg
    #protein_in_cell_outline = cell_outline * protein_seg
    
    Protein_Labels_void = ((protein_in_void))
    PROTEIN_INFO_void = skimage.measure.regionprops_table(Protein_Labels_void, properties = ('label','area'))
    PROTEIN_INFO_void=pd.DataFrame(PROTEIN_INFO_void)
    PROTEIN_INFO_void.columns = ['Protein_Label', 'Protein_Area_Pixels']
    PROTEIN_INFO_void['Void_Proten_Volume_Voxel_Resolved'] = PROTEIN_INFO_void['Protein_Area_Pixels'] * X_PIXEL_SIZE *Y_PIXEL_SIZE*Z_PIXEL_SIZE
    PROTEIN_INFO_void["ID"] = i
    PROTEIN_INFO_void.to_csv(out+i+"Protein_in_Void_Volume.csv")
    
    
    Protein_Labels = ((protein_in_g_cell))
    PROTEIN_INFO = skimage.measure.regionprops_table(Protein_Labels,  properties = ('label','area'))
    PROTEIN_INFO=pd.DataFrame(PROTEIN_INFO)
    PROTEIN_INFO.columns = ['Protein_Label', 'Protein_Area_Pixels']
    PROTEIN_INFO['gcell_Proten_Volume_Voxel_Resolved'] = PROTEIN_INFO['Protein_Area_Pixels'] * X_PIXEL_SIZE *Y_PIXEL_SIZE*Z_PIXEL_SIZE
    PROTEIN_INFO["ID"] = i
    PROTEIN_INFO.to_csv(out+i+"Protein_in_gcell_Volume.csv")
    
    Protein_Labels = ((protein_in_cell))
    PROTEIN_INFO = skimage.measure.regionprops_table(Protein_Labels,  properties = ('label','area'))
    PROTEIN_INFO=pd.DataFrame(PROTEIN_INFO)
    PROTEIN_INFO.columns = ['Protein_Label', 'Protein_Area_Pixels']
    PROTEIN_INFO['normal_cell_Proten_Volume_Voxel_Resolved'] = PROTEIN_INFO['Protein_Area_Pixels'] * X_PIXEL_SIZE *Y_PIXEL_SIZE*Z_PIXEL_SIZE
    PROTEIN_INFO["ID"] = i
    PROTEIN_INFO.to_csv(out+i+"Protein_in_cell_Volume.csv")
    
#     Protein_Labels = ((protein_in_cell_outline))
#     PROTEIN_INFO = skimage.measure.regionprops_table(Protein_Labels,  properties = ('label','area'))
#     PROTEIN_INFO=pd.DataFrame(PROTEIN_INFO)
#     PROTEIN_INFO.columns = ['Protein_Label', 'Protein_Area_Pixels']
#     PROTEIN_INFO['normal_cell_Proten_Volume_Voxel_Resolved'] = PROTEIN_INFO['Protein_Area_Pixels'] * X_PIXEL_SIZE *Y_PIXEL_SIZE*Z_PIXEL_SIZE
#     PROTEIN_INFO["ID"] = i
#     PROTEIN_INFO.to_csv(out+i+"protein_in_cell_outline.csv")
    
    
    g_CELL_INFO = skimage.measure.regionprops_table((gilal_cells_masked.astype(np.uint8)),  properties = ('label','area'))
    g_CELL_INFO=pd.DataFrame(g_CELL_INFO)
    g_CELL_INFO.columns = ['Cell_Label', 'Cell_Area_Pixels', ]
    g_CELL_INFO['g_Cell_Volume_Voxel_Resolved'] = g_CELL_INFO['Cell_Area_Pixels'] * X_PIXEL_SIZE *Y_PIXEL_SIZE*Z_PIXEL_SIZE
    g_CELL_INFO["ID"] = i
    g_CELL_INFO.to_csv(out+i+"g_Cell_Volume.csv")
    
    cell_label=label(cells_masked.astype(np.uint8))
    #cell_label=cell_outline *cell_AI
    skimage.io.imsave(out+i+"cell_label.tif", (cell_label))
    CELL_INFO = skimage.measure.regionprops_table((cell_label),  properties = ('label','area'))
    CELL_INFO=pd.DataFrame(CELL_INFO)
    CELL_INFO.columns = ['Cell_Label', 'Cell_Area_Pixels', ]
    CELL_INFO['normal_Cell_Volume_Voxel_Resolved'] = CELL_INFO['Cell_Area_Pixels'] * X_PIXEL_SIZE *Y_PIXEL_SIZE*Z_PIXEL_SIZE
    CELL_INFO["ID"] = i
    CELL_INFO.to_csv(out+i+"Cell_Volume.csv")
    
    VOID_INFO = skimage.measure.regionprops_table((void_binary.astype(np.uint8)),  properties = ('label','area'))
    VOID_INFO=pd.DataFrame(VOID_INFO)
    VOID_INFO.columns = ['Void_Label', 'Void_Area_Pixels', ]
    VOID_INFO['Void_Volume_Voxel_Resolved'] = VOID_INFO['Void_Area_Pixels'] * X_PIXEL_SIZE *Y_PIXEL_SIZE*Z_PIXEL_SIZE
    VOID_INFO["ID"] = i
    VOID_INFO.to_csv(out+i+"Void_Volume.csv")
    
    MASK_INFO = skimage.measure.regionprops_table((mask.astype(np.uint8)),  properties = ('label','area'))
    MASK_INFO=pd.DataFrame(MASK_INFO)
    MASK_INFO.columns = ['MASK_Label', 'MASK_Area_Pixels', ]
    MASK_INFO['MASK_Volume_Voxel_Resolved'] = MASK_INFO['MASK_Area_Pixels'] * X_PIXEL_SIZE *Y_PIXEL_SIZE*Z_PIXEL_SIZE
    MASK_INFO["ID"] = i
    MASK_INFO.to_csv(out+i+"MASK_Volume.csv")
    
    OUTLINE_INFO = skimage.measure.regionprops_table((cell_outline.astype(np.uint8)),  properties = ('label','area'))
    OUTLINE_INFO=pd.DataFrame(OUTLINE_INFO)
    OUTLINE_INFO.columns = ['OUTLINE_Label', 'OUTLINE_Area_Pixels', ]
    OUTLINE_INFO['OUTLINE_Volume_Voxel_Resolved'] = OUTLINE_INFO['OUTLINE_Area_Pixels'] * X_PIXEL_SIZE *Y_PIXEL_SIZE*Z_PIXEL_SIZE
    OUTLINE_INFO["ID"] = i
    OUTLINE_INFO.to_csv(out+i+"OUTLINE_Volume.csv")
    
    #lastly calc each protein

    
    protein_in_void_VOL = Volume(protein_in_void, spacing=(X_PIXEL_SIZE,Y_PIXEL_SIZE,Z_PIXEL_SIZE)) #sml
    protein_in_void_ISO = protein_in_void_VOL.isosurface(1)
    protein_in_void_ISO.write(out+i+"_protein_in_void.stl")
    
    protein_in_g_cell_VOL = Volume(protein_in_g_cell, spacing=(X_PIXEL_SIZE,Y_PIXEL_SIZE,Z_PIXEL_SIZE)) #sml
    protein_in_g_cell_ISO = protein_in_g_cell_VOL.isosurface(1)
    protein_in_g_cell_ISO.write(out+i+"_protein_in_g_cell.stl")
    
    protein_in_cell_VOL = Volume(protein_in_cell, spacing=(X_PIXEL_SIZE,Y_PIXEL_SIZE,Z_PIXEL_SIZE)) #sml
    protein_in_cell_ISO = protein_in_cell_VOL.isosurface(1)
    protein_in_cell_ISO.write(out+i+"_protein_in_cell.stl")
    
#     protein_in_cell_outline_VOL = Volume(protein_in_cell_outline, spacing=(X_PIXEL_SIZE,Y_PIXEL_SIZE,Z_PIXEL_SIZE)) #sml
#     protein_in_cell_outline_ISO = protein_in_cell_outline_VOL.isosurface(1)
#     protein_in_cell_outline_ISO.write(out+i+"_protein_in_cell_outline.stl")
    
    del (cell_label)
    del (protein_in_g_cell)
    del (protein_in_cell)
    del (mask)
    del (cell_outline)
    del (void_binary)
    del (cells)
    del (gilal_cells_masked)
    del (gilal_cells)
    del (gilal_cells_raw_masked)
    del (g_CELL_binary)
    del (protein)
    del (protein_in_void)
    del (cells_masked)
    del (CELL_binary)
    del (protein_mask)
    
    gc.collect()
    
    
    


In [ ]:
# #Create movie from base_path (search folders)
# from glob import glob
# import moviepy.video.io.ImageSequenceClip
# import os

# output_base = out_render
# fps=1

# image_files = [y for x in os.walk(output_base) for y in glob(os.path.join(x[0], '*.png'))]
# image_files = [word for word in image_files if 'RENDER' in word]
# clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(image_files, fps=fps)
# clip.write_videofile(output_base+'SOD_1_MOVIE.mp4')